In [0]:
#from pyspark.sql.functions import unbase64
from pyspark.sql.types import *
from pyspark.sql.functions import *
# Source with default settings
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

# Prepare training data from a list of (label, features) tuples.
training = spark.createDataFrame([
    (1.0, Vectors.dense([50.0, 20.0, 20.0])),
    (0.0, Vectors.dense([60.0, 30.0, 30.0])),
    (0.0, Vectors.dense([70, 25.0, 250.0])),
    (1.0, Vectors.dense([65, 18.0, 18.0]))], ["label", "features"])

# Create a LogisticRegression instance. This instance is an Estimator.
lr = LogisticRegression(maxIter=10, regParam=0.01)
# Print out the parameters, documentation, and any default values.
print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")

# Learn a LogisticRegression model. This uses the parameters stored in lr.
model1 = lr.fit(training)

# Since model1 is a Model (i.e., a transformer produced by an Estimator),
# we can view the parameters it used during fit().
# This prints the parameter (name: value) pairs, where names are unique IDs for this
# LogisticRegression instance.
print("Model 1 was fit using parameters: ")
print(model1.extractParamMap())

# We may alternatively specify parameters using a Python dictionary as a paramMap
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  # Specify 1 Param, overwriting the original maxIter.
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})  # Specify multiple Params.

# You can combine paramMaps, which are python dictionaries.
paramMap2 = {lr.probabilityCol: "myProbability"}  # Change output column name
paramMapCombined = paramMap.copy()
paramMapCombined.update(paramMap2)

# Now learn a new model using the paramMapCombined parameters.
# paramMapCombined overrides all parameters set earlier via lr.set* methods.
model2 = lr.fit(training, paramMapCombined)
print("Model 2 was fit using parameters: ")
print(model2.extractParamMap())



#connectionString = "Endpoint=sb://iothub-ns-ks-bigdata-5809426-7048d14abf.servicebus.windows.net/;SharedAccessKeyName=iothubowner;SharedAccessKey=sQQPHOQhXdvNxTBcBjTfz2fWOTTTnRwRZu+/9HKbXmM=;EntityPath=ks-bigdata-iot-hub-test"
connectionString = "Endpoint=sb://iothub-ns-soriothub-5891320-22f310a3d7.servicebus.windows.net/;SharedAccessKeyName=iothubowner;SharedAccessKey=8ifNKDo8d2yzt6r30nvvoL5vmA4pZspQFiiyTG/6400=;EntityPath=soriothub"


ehConf = {
  'eventhubs.connectionString' : connectionString
}

ehConf['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString)

# Schema of incoming data from IoT hub
schema = "timestamp timestamp, temperature double, humidity double, deviceId string, status string"

# Read directly from IoT Hub using the EventHubs library for Databricks
iot_stream = (
  spark.readStream.format("eventhubs")                                              # Read from IoT Hubs directly
    .options(**ehConf)                                                              # Use the Event-Hub-enabled connect 
    .load()                                                                         # Load the data
    .withColumn('reading', F.from_json(F.col('body').cast('string'), schema))       # Extract the "body" payload from  
)

#display(iot_stream)

iot_stream = iot_stream.withColumn('temperature', iot_stream.reading.temperature).withColumn('humidity', iot_stream.reading.humidity).withColumn('deviceId', iot_stream.reading.deviceId).withColumn('ts', iot_stream.reading.timestamp)
iot_stream2 = iot_stream.groupBy(window(iot_stream.ts, "5 seconds", "3 seconds"), iot_stream.deviceId).agg(avg('humidity'), avg('temperature'), min('humidity'), min('temperature'), max('humidity'), max('temperature'))

assembler = VectorAssembler(
    inputCols=["avg(humidity)", 'avg(temperature)', 'avg(temperature)'],
    outputCol="features")

features = assembler.transform(iot_stream2.withColumn('tsp', iot_stream2.window.start))
#print("Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'")
#output.select("features", "clicked").show(truncate=False)

predictions = model1.transform(features)
predictions.printSchema
display(predictions)



window,deviceId,avg(humidity),avg(temperature),min(humidity),min(temperature),max(humidity),max(temperature),tsp,features,rawPrediction,probability,prediction
"List(2020-11-12T19:04:24.000+0000, 2020-11-12T19:04:29.000+0000)",iot_sensor_test_3,70.21223212255286,26.379747396680894,65.84097741682082,22.037171107936757,79.03083371890958,31.60195225510066,2020-11-12T19:04:24.000+0000,"List(1, 3, List(), List(70.21223212255286, 26.379747396680894, 26.379747396680894))","List(1, 2, List(), List(-1.0592453766934007, 1.0592453766934007))","List(1, 2, List(), List(0.25745369060013773, 0.7425463093998623))",1.0
"List(2020-11-12T19:03:27.000+0000, 2020-11-12T19:03:32.000+0000)",iot_sensor_test_2,71.53751952622449,26.154619753415535,60.18391759929869,21.688868145589844,77.91346903457702,31.77272565731821,2020-11-12T19:03:27.000+0000,"List(1, 3, List(), List(71.53751952622449, 26.154619753415535, 26.154619753415535))","List(1, 2, List(), List(-1.3299020677521192, 1.3299020677521192))","List(1, 2, List(), List(0.20917556481652888, 0.7908244351834711))",1.0
"List(2020-11-12T19:04:27.000+0000, 2020-11-12T19:04:32.000+0000)",iot_sensor_test_7,69.92445957593063,30.360330322751047,69.92445957593063,30.360330322751047,69.92445957593063,30.360330322751047,2020-11-12T19:04:27.000+0000,"List(1, 3, List(), List(69.92445957593063, 30.360330322751047, 30.360330322751047))","List(1, 2, List(), List(1.4572954738461572, -1.4572954738461572))","List(1, 2, List(), List(0.8111186760012821, 0.18888132399871796))",0.0
"List(2020-11-12T19:06:27.000+0000, 2020-11-12T19:06:32.000+0000)",iot_sensor_test_0,75.905309063801,30.66317997452296,74.20724248891725,26.248738676273593,76.81594896712508,33.6509567850839,2020-11-12T19:06:27.000+0000,"List(1, 3, List(), List(75.905309063801, 30.66317997452296, 30.66317997452296))","List(1, 2, List(), List(1.0602743353323696, -1.0602743353323696))","List(1, 2, List(), List(0.7427429676505888, 0.2572570323494111))",0.0
"List(2020-11-12T19:02:15.000+0000, 2020-11-12T19:02:20.000+0000)",iot_sensor_test_7,77.88747202490948,29.328109022045854,75.85686093625944,27.980995027357196,79.91808311355953,30.675223016734513,2020-11-12T19:02:15.000+0000,"List(1, 3, List(), List(77.88747202490948, 29.328109022045854, 29.328109022045854))","List(1, 2, List(), List(0.03137621742413721, -0.03137621742413721))","List(1, 2, List(), List(0.5078434109016455, 0.4921565890983544))",0.0
"List(2020-11-12T19:03:00.000+0000, 2020-11-12T19:03:05.000+0000)",iot_sensor_test_9,70.72714195884356,25.092470985165317,63.65782876625609,20.7349815907013,78.61621688585728,34.11410289106196,2020-11-12T19:03:00.000+0000,"List(1, 3, List(), List(70.72714195884356, 25.092470985165317, 25.092470985165317))","List(1, 2, List(), List(-1.9144229593366884, 1.9144229593366884))","List(1, 2, List(), List(0.12848477057972027, 0.8715152294202797))",1.0
"List(2020-11-12T19:01:42.000+0000, 2020-11-12T19:01:47.000+0000)",iot_sensor_test_9,70.34818840597703,27.706270192122915,63.479511274329965,20.10384585116352,78.093924463581,34.3308687341315,2020-11-12T19:01:42.000+0000,"List(1, 3, List(), List(70.34818840597703, 27.706270192122915, 27.706270192122915))","List(1, 2, List(), List(-0.24334242106845494, 0.24334242106845494))","List(1, 2, List(), List(0.4394628287349641, 0.5605371712650359))",1.0
"List(2020-11-12T19:01:21.000+0000, 2020-11-12T19:01:26.000+0000)",iot_sensor_test_7,69.49813382006393,31.53691707226757,69.49813382006393,31.53691707226757,69.49813382006393,31.53691707226757,2020-11-12T19:01:21.000+0000,"List(1, 3, List(), List(69.49813382006393, 31.53691707226757, 31.53691707226757))","List(1, 2, List(), List(2.23459468775693, -2.23459468775693))","List(1, 2, List(), List(0.9033133955603029, 0.09668660443969711))",0.0
"List(2020-11-12T19:06:36.000+0000, 2020-11-12T19:06:41.000+0000)",iot_sensor_test_5,66.48033208393863,26.728656422211287,60.47553814085041,20.80542576639527,71.86325779332178,30.991889316116662,2020-11-12T19:06:36.000+0000,"List(1, 3, List(), List(66.4803320